In [1]:
# Python libraries
import numpy as np
import pandas as pd
# import datetime

# 3rd party libraries
import ness

# Internal imports
from data_manager import to_numeric, create_agg_var, two_date_cols_diff, days_from_today, last_row, create_unpaid_cols

In [2]:
pd.options.display.max_columns = 200
# pd.options.display.max_rows = 10000

In [3]:
# Set the ness parameters
ness_parameters = {
    'bucket':"data.postpay.io",
    'key':"api",
    'profile':"default"
}
# Instantiate the data lake
dl = ness.dl(**ness_parameters)
# Read/Sync all the tables from the data lake
dl.sync()
# Create data frames from the tables
cart_df = dl.read("cart")
orders_df = dl.read("orders")
customers_df = dl.read("customers")
addresses_df = dl.read("addresses")
instalment_plans_df = dl.read("instalment-plans")
instalments_df = dl.read("instalments")
transactions_df = dl.read("transactions")
refunds_df = dl.read("refunds")

In [4]:
# Convert the anount columns to numeric
to_numeric(df=instalment_plans_df, columns_list=['total_amount'])

In [5]:
# Filter customers_df to get only the relevant variables
filtered_customers_df = customers_df[['customer_id', 'created', 'date_of_birth']].rename(columns={'created':'customer_first_joined'})

In [6]:
filtered_customers_df

,customer_id,customer_first_joined,date_of_birth
0,17374,2021-08-08 12:42:31.648990,1988-11-23
1,35034,2021-10-30 03:09:34.243808,1990-01-01
2,40723,2021-11-11 18:28:20.796900,1989-08-24
3,16165,2021-07-26 17:58:15.206670,None
4,24307,2021-09-24 19:12:44.714127,None
...,...,...,...
48563,18678,2021-08-23 11:41:21.459482,1982-08-23
48564,9060,2021-04-29 14:42:22.976979,1989-05-21
48565,22249,2021-09-14 06:04:52.872019,1982-09-03
48566,37455,2021-11-05 04:30:48.523456,1960-01-01


In [7]:
# Create AOV df
aov_df = create_agg_var(
    df=instalment_plans_df,
    groupby_col='customer_id',
    orig_cols=['total_amount'],
    new_col_names=['avg_order_value'],
    agg_fnc='mean'
    )

In [46]:
tmp_tmp = instalment_plans_df[pd.notnull(instalment_plans_df['customer_id'])][['customer_id', 'created', 'total_amount']]

In [47]:
tmp_tmp['rn'] = tmp_tmp.groupby(["customer_id"])["created"].rank(method="first", ascending=True)

In [53]:
tmp_tmp_sort = tmp_tmp.sort_values(["customer_id", "rn"])

In [54]:
tmp_tmp_sort

,customer_id,created,total_amount,rn
48044,64.0,2021-10-31 10:02:57.280467,91.00,1.0
47597,64.0,2021-11-01 11:44:17.953913,23.00,2.0
47598,64.0,2021-11-01 12:24:36.978093,46.41,3.0
21063,68.0,2020-08-10 08:52:34.657379,131.00,1.0
21062,68.0,2020-08-24 17:42:33.779159,147.00,2.0
...,...,...,...,...
19692,48937.0,2021-11-24 23:29:15.860557,4470.00,1.0
24707,48939.0,2021-11-24 23:34:36.191598,684.90,1.0
6625,48942.0,2021-11-24 23:47:56.029899,285.00,1.0
35472,48944.0,2021-11-24 23:56:38.354025,1086.80,1.0


In [61]:
for i in tmp_tmp_sort['customer_id'].unique():
    for d in tmp_tmp_sort[tmp_tmp_sort['customer_id'] == i]['rn']:
        print(i, d, tmp_tmp_sort[(tmp_tmp_sort['customer_id'] == i) & (tmp_tmp_sort['rn'] < d)]['total_amount'].mean())
    if d == 3:
        break

64.0 1.0 nan
64.0 2.0 91.0
64.0 3.0 57.0


In [ ]:
tips.groupby(["day"])["total_bill"].rank(
            method="first", ascending=False

In [32]:
tmp_tmp['avg_amnt'] = tmp_tmp.groupby(by=['customer_id', 'created']).transform(np.mean)

/var/folders/5b/q1b1f10s6p39_ns07cll2dxh0000gn/T/ipykernel_62538/1882369050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_tmp['avg_amnt'] = tmp_tmp.groupby(by=['customer_id', 'created']).transform(np.mean)


In [33]:
tmp_tmp[tmp_tmp['customer_id'] == 64]

,customer_id,created,total_amount,avg_amnt
47597,64.0,2021-11-01 11:44:17.953913,23.00,23.00
47598,64.0,2021-11-01 12:24:36.978093,46.41,46.41
48044,64.0,2021-10-31 10:02:57.280467,91.00,91.00


In [15]:
tmp_aov

,customer_id,created,total_amount
0,64.0,2021-10-31 10:02:57.280467,91.00
1,64.0,2021-11-01 11:44:17.953913,23.00
2,64.0,2021-11-01 12:24:36.978093,46.41
3,68.0,2020-08-10 08:52:34.657379,131.00
4,68.0,2020-08-24 17:42:33.779159,147.00
...,...,...,...
44784,48937.0,2021-11-24 23:29:15.860557,4470.00
44785,48939.0,2021-11-24 23:34:36.191598,684.90
44786,48942.0,2021-11-24 23:47:56.029899,285.00
44787,48944.0,2021-11-24 23:56:38.354025,1086.80


In [17]:
for i in tmp_aov['customer_id'].unique():
    for d in tmp_aov[tmp_aov['customer_id'] == i]['created']:
        print(i,d)

64.0 2021-10-31 10:02:57.280467
64.0 2021-11-01 11:44:17.953913
64.0 2021-11-01 12:24:36.978093
68.0 2020-08-10 08:52:34.657379
68.0 2020-08-24 17:42:33.779159
68.0 2020-09-23 17:53:16.700152
68.0 2020-10-22 04:03:08.380870
68.0 2020-11-26 05:12:05.037328
68.0 2020-11-26 07:45:17.106456
68.0 2020-12-12 10:04:43.039085
68.0 2020-12-20 17:09:03.268752
68.0 2020-12-22 04:13:18.585247
68.0 2020-12-30 09:31:02.959019
68.0 2021-02-02 18:16:01.287941
68.0 2021-02-10 04:03:40.782565
68.0 2021-02-15 06:47:03.789912
68.0 2021-02-22 05:32:41.626803
68.0 2021-03-11 19:16:35.193069
68.0 2021-03-22 08:08:12.476035
68.0 2021-03-27 15:07:28.983873
68.0 2021-04-05 16:56:13.197691
68.0 2021-06-05 18:35:11.669945
68.0 2021-06-14 19:49:59.043486
68.0 2021-08-16 11:36:38.735127
68.0 2021-08-17 11:50:49.315182
68.0 2021-08-29 07:05:56.170522
68.0 2021-09-04 12:34:21.736695
68.0 2021-10-14 11:23:41.575354
68.0 2021-10-15 11:42:15.219299
68.0 2021-10-16 11:00:57.561141
68.0 2021-11-01 09:59:10.839847
68.0 202

In [10]:
instalment_plans_df.head()

,created,payment_method_fingerprint,downpayment_amount,billing_address_id,device_fingerprint,total_downpayment,checkout_completed,payment_method_brand,plan,payment_method_expires,shipping_address_id,total_amount,merchant_id,customer_id,currency,gateway_name,id_number,checkout_verified,cancelled,payment_method_country,shipping_amount,reference,merchant_name,completed,instalment_plan_id,payment_interval,customer_date_joined,status_changed,order_id,checkout_type,customer_email,status,num_instalments,shipping_id,customer_blacklisted,payment_method_type,date_of_birth,downpayment_refunded_amount,user_agent,phone,ip_address,transaction_cost_rate,transaction_cost_amount
0,2021-11-11 13:40:22.453145,c2168ba8f48b4b728b1a8db201b7c7fe,541.67,72920.0,973ba9e918711784386cbe3b473bbb1c,541.67,2021-11-11 13:40:18.859538,visa,funded,2026-03-31,72920.0,1625.00,209,40500.0,AED,checkoutV2,784199157917255,2021-11-11 13:37:33.945995,NaT,AE,0.00,295832984214082296192877,Ounass,NaT,53500,NaN,NaT,2021-11-11 13:40:33.557085,103444,seamless,sara.j.hamdan@gmail.com,captured,3,62319.0,NaT,debit,None,0.00,Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_1 like...,971581771646,91.75.215.152,0.022,0.5
1,2021-07-24 12:24:25.953241,0bbd516d9d3841c78db2121a06a73f8c,91.67,34807.0,ba606ad5da0f6b0d9e722d38a7d00a13,91.67,2021-07-24 12:23:39.643465,visa,funded,2026-07-31,34807.0,275.00,214,15950.0,AED,checkoutV2,784197410865048,2021-07-24 12:22:37.474975,NaT,AE,0.00,FLAEHDE0605147-60fc05e3736b7,Footlocker,2021-09-24 12:33:47.692180,25161,NaN,2021-07-23 22:12:36,2021-07-24 12:24:34.530518,47087,default,magnoelizer@gmail.com,captured,3,27563.0,NaT,credit,1974-01-30,0.00,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,971566519794,5.30.141.67,0.022,0.5
2,2021-09-11 11:58:44.413922,72280bb4d5784f789e11e65c82ebd24d,130.05,45861.0,425db8d63ca25389de6338f964b168b8,130.05,2021-09-11 11:58:39.611180,visa,funded,2026-02-28,45861.0,390.15,279,21820.0,AED,checkoutV2,784199058321326,2021-09-11 11:57:07.823328,NaT,AE,0.00,PKAEHDE0088425-613c998139e6e,Pottery Barn Kids,2021-11-11 12:00:31.743115,32819,NaN,2021-09-04 13:41:24,2021-09-11 11:58:53.932230,63045,seamless,ahmed.mohmd.a@gmail.com,captured,3,37273.0,NaT,debit,None,0.00,Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like...,971568385050,2001:8f8:132f:df93:1c59:7961:24d0:63f1,0.022,0.5
3,2021-07-07 15:55:48.023523,ba8016199afb4e5d99a37e3b17629ba9,45.00,32068.0,51141d7a672e7afe8a18a7d34cbe162f,45.00,2021-07-07 15:55:48.150700,visa,funded,2025-08-31,NaN,45.00,181,NaN,AED,checkoutV2,None,NaT,NaT,AE,None,30604-60e5ce6bcce84,KAVE The Story of things,2021-07-07 15:55:58.759786,23108,NaN,2021-07-07 11:55:23,2021-07-07 15:55:58.762953,43444,default,farahhassan77@hotmail.com,captured,1,NaN,NaT,debit,None,0.00,Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like M...,None,185.97.92.116,0.022,0.5
4,2021-08-28 16:47:01.306260,72b023f5c9f24584b32931403a81a9be,216.30,36163.0,669afb65094728cb844b59fd8113a226,216.30,2021-08-28 16:47:01.291113,mastercard,funded,2026-06-30,36163.0,216.30,256,NaN,AED,checkoutV2,None,NaT,NaT,AE,0.00,4050247254196,Ecogardenz,2021-08-28 16:47:02.607515,30010,NaN,2021-06-08 09:20:41,2021-08-28 16:47:02.610584,56945,default,lizy.thomson@gmail.com,captured,1,33481.0,NaT,debit,None,0.00,Mozilla/5.0 (Linux; Android 10; SM-G965F) Appl...,None,94.204.177.218,0.022,0.5


In [8]:
aov_df

,customer_id,avg_order_value
0,64.0,53.470000
1,68.0,812.751613
2,71.0,817.659615
3,73.0,2292.840000
4,76.0,940.280000
...,...,...
29796,48937.0,4470.000000
29797,48939.0,684.900000
29798,48942.0,285.000000
29799,48944.0,1086.800000


In [7]:
# Create sum of fees per order
fees_df = create_agg_var(
    df=instalments_df,
    groupby_col='instalment_plan_id',
    orig_cols=['penalty_fee'],
    new_col_names=['sum_fees_per_order'],
    agg_fnc='sum',
    )

cid_fees_df = fees_df.merge(instalment_plans_df[['instalment_plan_id', 'customer_id', 'created']], how='left', on='instalment_plan_id')

In [8]:
# Create avg fees variables
fees_365d_df = create_agg_var(
    df=cid_fees_df,
    groupby_col='customer_id',
    orig_cols=['sum_fees_per_order'],
    new_col_names=['avg_fees_per_order_365d'],
    agg_fnc='mean',
    date_filter_col = 'created',
    nr_days_filter = 365
    )

fees_180d_df = create_agg_var(
    df=cid_fees_df,
    groupby_col='customer_id',
    orig_cols=['sum_fees_per_order'],
    new_col_names=['avg_fees_per_order_180d'],
    agg_fnc='mean',
    date_filter_col = 'created',
    nr_days_filter = 180
    )

fees_90d_df = create_agg_var(
    df=cid_fees_df,
    groupby_col='customer_id',
    orig_cols=['sum_fees_per_order'],
    new_col_names=['avg_fees_per_order_90d'],
    agg_fnc='mean',
    date_filter_col = 'created',
    nr_days_filter = 90
    )

fees_30d_df = create_agg_var(
    df=cid_fees_df,
    groupby_col='customer_id',
    orig_cols=['sum_fees_per_order'],
    new_col_names=['avg_fees_per_order_30d'],
    agg_fnc='mean',
    date_filter_col = 'created',
    nr_days_filter = 30
    )

In [9]:
# Create the number of merchants per customer
nr_merchants_df = create_agg_var(
    df=instalment_plans_df,
    groupby_col='customer_id',
    orig_cols=['merchant_name'],
    new_col_names=['count_merchants_per_customer'],
    agg_fnc='nunique'
    )

In [10]:
# Create the number of open orders
cid_status_df = instalment_plans_df[['instalment_plan_id', 'customer_id']].merge(instalments_df[['instalment_plan_id', 'status', 'total']], how='left', on='instalment_plan_id')
cid_status_due_df = cid_status_df[cid_status_df['status'] == 'due']
open_orders_df = create_agg_var(
    df=cid_status_due_df,
    groupby_col='customer_id',
    orig_cols=['instalment_plan_id'],
    new_col_names=['count_open_orders'],
    agg_fnc='nunique'
    )

# Create the number of paid orders
cid_status_paid_df = cid_status_df[cid_status_df['status'] == 'paid']
paid_orders_df = create_agg_var(
    df=cid_status_paid_df,
    groupby_col='customer_id',
    orig_cols=['instalment_plan_id'],
    new_col_names=['count_paid_orders'],
    agg_fnc='nunique'
    )

# Create the number of unpaid orders
cid_status_unpaid_df = cid_status_df[cid_status_df['status'] == 'unpaid']
unpaid_orders_df = create_agg_var(
    df=cid_status_unpaid_df,
    groupby_col='customer_id',
    orig_cols=['instalment_plan_id'],
    new_col_names=['count_unpaid_orders'],
    agg_fnc='nunique'
    )

# The number of paid instalments
cid_status_paid_df = cid_status_df[cid_status_df['status'] == 'paid']
paid_inst_df = create_agg_var(
    df=cid_status_paid_df,
    groupby_col='customer_id',
    orig_cols=['status'],
    new_col_names=['count_paid_instalments'],
    agg_fnc='count'
    )

# The number of unpaid instalments
cid_status_unpaid_df = cid_status_df[cid_status_df['status'] == 'unpaid']
unpaid_inst_df = create_agg_var(
    df=cid_status_unpaid_df,
    groupby_col='customer_id',
    orig_cols=['status'],
    new_col_names=['count_unpaid_instalments'],
    agg_fnc='count'
    )

# The sum of outstanding captured debt
cid_status_due_df = cid_status_df[cid_status_df['status'] == 'due']
current_exposure_df = create_agg_var(
    df=cid_status_due_df,
    groupby_col='customer_id',
    orig_cols=['total'],
    new_col_names=['current_exposure'],
    agg_fnc='sum'
    )

# The sum of all oders amount
cid_status_paid_df = cid_status_df[cid_status_df['status'] == 'paid']
sum_paid_df = create_agg_var(
    df=cid_status_paid_df,
    groupby_col='customer_id',
    orig_cols=['total'],
    new_col_names=['sum_paid_amount'],
    agg_fnc='sum'
    )

In [28]:
cid_scheduled_status_df = instalment_plans_df[['instalment_plan_id', 'customer_id']].merge(instalments_df[['instalment_plan_id', 'scheduled', 'status', 'total']], how='left', on='instalment_plan_id')
cid_scheduled_status_unpaid_df = cid_scheduled_status_df[cid_status_df['status'] == 'unpaid']
max_scheduled_unpaid_df = create_agg_var(
    df=cid_scheduled_status_unpaid_df,
    groupby_col='customer_id',
    orig_cols=['scheduled'],
    new_col_names=['max_scheduled'],
    agg_fnc='max'
    )
days_from_today(
    df = max_scheduled_unpaid_df,
    new_col_name = 'days_since_last_unpaid',
    date_col = 'max_scheduled',
    drop_orig_col = True
    )

In [43]:
max_scheduled_unpaid_df

,customer_id,days_since_last_unpaid
0,128.0,399
1,338.0,188
2,383.0,272
3,465.0,377
4,598.0,6
...,...,...
786,31270.0,1
787,31272.0,1
788,31306.0,1
789,31318.0,1


In [35]:
behaviour_df = (filtered_customers_df
.merge(aov_df, how='left', on='customer_id') # Nulls -> Remove these rows
.merge(fees_365d_df, how='left', on='customer_id') # Nulls -> 0
.merge(fees_180d_df, how='left', on='customer_id') # Nulls -> 0
.merge(fees_90d_df, how='left', on='customer_id') # Nulls -> 0
.merge(fees_30d_df, how='left', on='customer_id') # Nulls -> 0
.merge(nr_merchants_df, how='left', on='customer_id') # Nulls -> Remove these rows
.merge(open_orders_df, how='left', on='customer_id') # Nulls -> 0
.merge(paid_orders_df, how='left', on='customer_id') # Nulls -> 0
.merge(unpaid_orders_df, how='left', on='customer_id') # Nulls -> 0
.merge(paid_inst_df, how='left', on='customer_id') # Nulls -> 0
.merge(unpaid_inst_df, how='left', on='customer_id') # Nulls -> 0
.merge(current_exposure_df, how='left', on='customer_id') # Nulls -> 0
.merge(sum_paid_df, how='left', on='customer_id') # Nulls -> 0
.merge(max_scheduled_unpaid_df, how='left', on='customer_id') # Nulls -> 0
)

In [37]:
behaviour_df[behaviour_df['customer_id'] == 14042]

,customer_id,customer_first_joined,date_of_birth,avg_order_value,avg_fees_per_order_365d,avg_fees_per_order_180d,avg_fees_per_order_90d,avg_fees_per_order_30d,count_merchants_per_customer,count_open_orders,count_paid_orders,count_unpaid_orders,count_paid_instalments,count_unpaid_instalments,current_exposure,sum_paid_amount,days_since_last_unpaid
0,14042,2021-07-02 09:55:40.210638,1971-06-24,1720.44,0.0,0.0,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,NaN,1146.96,NaN


In [38]:
instalment_plans_df['status'].value_counts()

captured    49993
Name: status, dtype: int64